In [1]:
import Pkg.instantiate
instantiate()

In [2]:
using BenchmarkTools: @btime, @belapsed, @ballocated
using LoopVectorization: @turbo

In [3]:
using LinearAlgebra: mul!

In [4]:
using CairoMakie

In [40]:
function add_to_A_B_times_C!(A, B, C)
    @turbo for j in axes(C, 2)
        for k in axes(B, 2)
            for i in axes(A, 1)
                A[i, j] += B[i, k] * C[k, j]
            end
        end
    end
    return A
end

function add_to_A_B_times_C!(A, B, C, bks)
    variable = zeros(7, 2)
    for j in 1:ceil(Int, size(C, 2)/bks)
        # println('j', j)
        for k in 1:ceil(Int, size(B, 2)/bks)
            # print('k', k)
            for i in 1:ceil(Int, size(A, 1)/bks)
                # print('i', i)
                for block_j in Int((j-1)*bks):min(Int((j)*bks-1), size(C, 2)-1)
                    # println()
                    # print(' ', 'b', ' ', 'j', block_j)
                    for block_k in Int((k-1)*bks):min(Int((k)*bks-1), size(B, 2)-1)
                        # print(' ', 'k', block_k)
                        for block_i in Int((i-1)*bks):min(Int((i)*bks-1), size(A, 1)-1)
                            # print(' ', 'i', block_i)
                            A[block_i+1, block_j+1] += B[block_i+1, block_k+1] * C[block_k+1, block_j+1]
                        end
                    end
                end
                # println()
            end
        end
    end
    return(A)
end

function oblivious_add_to_A_B_times_C!(A, B, C, bks)
    i_size = size(A, 1)
    j_size = size(C, 2)
    k_size = size(B, 2)

    i_size = div(i_size,2)
    j_size = div(j_size,2)
    k_size = div(k_size,2)

    # println(i_size, j_size, k_size)
    # display(A)
    # display(B)
    # display(C)

    # display(B[1:i_size, 1:k_size])
    # display(B[i_size+1:end, 1:k_size])
    # display(B[1:i_size, k_size+1:end])
    # display(B[i_size+1:end, k_size+1:end])

    # display(A)
    display(B)
    display(C)
    # If we want to further subdivide
    if min(i_size, j_size, k_size) > bks
        println('Y','E','S', 1)
        A[1:i_size, 1:j_size] += oblivious_add_to_A_B_times_C!(
            A[1:i_size, 1:j_size],
            B[1:i_size, 1:k_size],
            C[1:k_size, 1:j_size],
            bks
        )
        display(A)
        println('Y','E','S', 2)
        A[1:i_size, j_size+1:end] += oblivious_add_to_A_B_times_C!(
            A[1:i_size, j_size+1:end],
            B[1:i_size, k_size+1:end],
            C[1:k_size, j_size+1:end],
            bks
        )
        display(A)
        println('Y','E','S', 3)
        A[i_size+1:end, 1:j_size] += oblivious_add_to_A_B_times_C!(
            A[i_size+1:end, 1:j_size],
            B[i_size+1:end, 1:k_size],
            C[k_size+1:end, 1:j_size],
            bks
        )
        display(A)
        println('Y','E','S', 4)
        A[i_size+1:end, j_size+1:end] += oblivious_add_to_A_B_times_C!(
            A[i_size+1:end, j_size+1:end],
            B[i_size+1:end, k_size+1:end],
            C[k_size+1:end, j_size+1:end],
            bks
        )
        display(A)
    #If we are ready to break the recursion
    else
        println('N','O')
        return add_to_A_B_times_C!(A, B, C)
    end
end

oblivious_add_to_A_B_times_C! (generic function with 1 method)

In [42]:
# Testing blocked matmul function
A = zeros(12, 6)
B = rand(1:6, 12, 6)
C = rand(1:6, 6, 6)
# println(A)
# println(B)
# println(C)
# println("Testing for memory allocation of blocked matmul...")
# allocated_memory = @ballocated add_to_A_B_times_C!(A, B, C, 301)
# @assert allocated_memory == 0
# println("No memory allocated, good!")
# println("Testing correctness of blocked matmul...")
A .= 0; 
# println()
# display(A)
oblivious_add_to_A_B_times_C!(A, B, C, 2)
display(A)
display(B*C)
# display(A)

@assert A ≈ B * C
println("Result correct, good!")

12×6 Matrix{Int64}:
 1  3  5  3  4  6
 5  2  6  6  2  5
 6  2  4  6  1  5
 1  3  3  6  4  5
 4  6  3  2  5  4
 4  5  1  4  1  6
 5  2  4  4  4  3
 6  6  1  6  4  6
 2  6  6  6  1  2
 6  4  1  2  5  5
 6  2  1  2  3  1
 4  5  3  2  2  1

6×6 Matrix{Int64}:
 1  1  6  4  2  3
 1  2  1  1  2  4
 1  5  4  5  2  3
 4  3  5  4  1  5
 4  6  2  1  1  1
 1  6  1  1  6  6

YES1


6×3 Matrix{Int64}:
 1  3  5
 5  2  6
 6  2  4
 1  3  3
 4  6  3
 4  5  1

3×3 Matrix{Int64}:
 1  1  6
 1  2  1
 1  5  4

...

NO


12×6 Matrix{Float64}:
  9.0  32.0  29.0  46.0  26.0  43.0
 13.0  39.0  56.0  51.0  26.0  41.0
 12.0  30.0  54.0  50.0  24.0  37.0
  7.0  22.0  21.0  53.0  30.0  49.0
 13.0  31.0  42.0  33.0  22.0  38.0
 10.0  19.0  33.0  47.0  22.0  34.0
 32.0  51.0  33.0  23.0  26.0  42.0
 49.0  60.0  43.0  34.0  46.0  70.0
 38.0  78.0  28.0  27.0  19.0  43.0
 41.0  48.0  39.0  18.0  37.0  45.0
 33.0  36.0  35.0  12.0  11.0  19.0
 39.0  60.0  33.0  11.0  10.0  18.0

12×6 Matrix{Float64}:
  9.0  32.0  29.0  46.0  26.0  43.0
 13.0  39.0  56.0  51.0  26.0  41.0
 12.0  30.0  54.0  50.0  24.0  37.0
  7.0  22.0  21.0  53.0  30.0  49.0
 13.0  31.0  42.0  33.0  22.0  38.0
 10.0  19.0  33.0  47.0  22.0  34.0
 32.0  51.0  33.0  23.0  26.0  42.0
 49.0  60.0  43.0  34.0  46.0  70.0
 38.0  78.0  28.0  27.0  19.0  43.0
 41.0  48.0  39.0  18.0  37.0  45.0
 33.0  36.0  35.0  12.0  11.0  19.0
 39.0  60.0  33.0  11.0  10.0  18.0

12×6 Matrix{Int64}:
 43  101  58  54  61   85
 50   99  95  83  64  103
 45   84  91  76  61   99
 52   94  64  55  54   88
 45   91  66  54  57   84
 36   73  61  49  61   92
 46   83  79  65  48   77
 59  101  90  69  72  115
 44   80  76  71  47   91
 44   85  69  51  59   82
 30   45  59  43  29   48
 29   53  56  47  34   59

LoadError: AssertionError: A ≈ B * C

In [ ]:
# Testing blocked matmul function
A = zeros(6, 2)
B = rand(1:6, 6, 4)
C = rand(1:6, 4, 2)
# println(A)
# println(B)
# println(C)
# println("Testing for memory allocation of blocked matmul...")
# allocated_memory = @ballocated add_to_A_B_times_C!(A, B, C, 301)
# @assert allocated_memory == 0
# println("No memory allocated, good!")
# println("Testing correctness of blocked matmul...")
A .= 0; 
# println()
println(add_to_A_B_times_C!(A, B, C, 2))
println(B*C)

@assert A ≈ B * C
println("Result correct, good!")

In [ ]:
# Testing blocked matmul function
A = zeros(2014, 301)
B = randn(2014, 1037)
C = randn(1037, 301)
println("Testing for memory allocation of blocked matmul...")
allocated_memory = @ballocated add_to_A_B_times_C!(A, B, C, 301)
print(allocated_memory)
@assert allocated_memory == 0
println("No memory allocated, good!")
println("Testing correctness of blocked matmul...")
A .= 0; add_to_A_B_times_C!(A, B, C, 301)
@assert A ≈ B * C
println("Result correct, good!")

In [ ]:
A = rand(5, 4)
print(A)
A[1:3, 2:4]


In [ ]:
A = [1, 2, 3]
for i in Int(1:1.5)
    println(A[i])
end

In [ ]:
x =5
x/=2
